In [1]:
%run helper_functions.ipynb

In [2]:
DRY_RUN = False

In [3]:
import pandas as pd

In [4]:
%%time

if not DRY_RUN:
    transactions = pd.read_parquet('data/transactions_train.parquet')
    customers = pd.read_parquet('data/customers.parquet')
    articles = pd.read_parquet('data/articles.parquet')
else:
    sample = 0.05
    transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
    customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
    articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 2.38 s, sys: 986 ms, total: 3.37 s
Wall time: 1.43 s


In [5]:
transactions.week


25784         0
25785         0
5389          0
5390          0
47429         0
           ... 
31774722    104
31774723    104
31779097    104
31779098    104
31780475    104
Name: week, Length: 31788324, dtype: int8

In [6]:
#test_week = transactions.week.max()
test_week = 97


In [7]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0
...,...,...,...,...,...,...
31774722,2020-09-22,18439937050817258297,891591003,0.084729,2,104
31774723,2020-09-22,18439937050817258297,869706005,0.084729,2,104
31779097,2020-09-22,18440902715633436014,918894002,0.016932,1,104
31779098,2020-09-22,18440902715633436014,761269001,0.016932,1,104


In [8]:
#transactions = transactions[transactions.week > test_week - 10]
transactions = transactions[transactions.week > 83]
transactions = transactions[transactions.week < test_week]


In [9]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
25231189,2020-04-29,2319521627154817,879436002,0.038119,2,84
25231190,2020-04-29,2319521627154817,814766002,0.030492,2,84
25231191,2020-04-29,2319521627154817,843753004,0.040017,2,84
25211825,2020-04-29,4233235614030232,858052007,0.022864,2,84
25211826,2020-04-29,4233235614030232,852577002,0.045746,2,84
...,...,...,...,...,...,...
29567770,2020-07-28,18445692600011570228,802979001,0.022017,2,96
29567771,2020-07-28,18445692600011570228,832330003,0.016932,2,96
29567772,2020-07-28,18445692600011570228,832330003,0.016932,2,96
29567773,2020-07-28,18445692600011570228,510283001,0.010153,2,96


# Generating candidates

The records I generate below can both be used as candidates for evaluation by our ranker (for the test week) and as negative examples (for training data).

I am going with the name candidates for both of these use cases, but that is certainly not ideal. Feel free to refactor the naming to increase the clarity of your code.

### Last purchase candidates

In [10]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 18.2 s, sys: 169 ms, total: 18.3 s
Wall time: 18.4 s


In [11]:
c2weeks

customer_id
28847241659200                      [86, 91, 92, 94, 95, 96]
41318098387474                                      [87, 91]
77117344919861                                      [84, 90]
200292573348128         [84, 85, 86, 87, 90, 92, 93, 95, 96]
208119717816961                                         [94]
                                        ...                 
18446522881501826039                                    [96]
18446566209623725451                                    [92]
18446571879212697038                            [88, 92, 95]
18446624797007271432                                    [95]
18446737527580148316                                    [84]
Name: week, Length: 539218, dtype: object

In [12]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 676 ms, sys: 58.7 ms, total: 734 ms
Wall time: 739 ms


In [13]:
c2weeks2shifted_weeks

{28847241659200: {86: 91, 91: 92, 92: 94, 94: 95, 95: 96, 96: 97},
 41318098387474: {87: 91, 91: 97},
 77117344919861: {84: 90, 90: 97},
 200292573348128: {84: 85,
  85: 86,
  86: 87,
  87: 90,
  90: 92,
  92: 93,
  93: 95,
  95: 96,
  96: 97},
 208119717816961: {94: 97},
 248294615847351: {91: 96, 96: 97},
 259325451862100: {85: 97},
 272412481300040: {84: 86, 86: 91, 91: 95, 95: 96, 96: 97},
 302826759708653: {90: 97},
 345001598676045: {92: 97},
 366493139417506: {96: 97},
 375055163245029: {96: 97},
 394822477655645: {90: 97},
 441030060120011: {91: 97},
 477147891906489: {87: 97},
 649760207043851: {89: 96, 96: 97},
 690285180337957: {84: 87, 87: 90, 90: 96, 96: 97},
 736218475114453: {96: 97},
 745180086074610: {87: 93, 93: 96, 96: 97},
 762483386043116: {92: 97},
 857913002275398: {90: 92, 92: 94, 94: 95, 95: 96, 96: 97},
 964326548579219: {87: 97},
 1037449031262554: {88: 92, 92: 93, 93: 94, 94: 97},
 1083424902212452: {93: 97},
 1126567349020901: {87: 89, 89: 97},
 11299412879

In [14]:
candidates_last_purchase = transactions.copy()

In [15]:
candidates_last_purchase


,t_dat,customer_id,article_id,price,sales_channel_id,week
25231189,2020-04-29,2319521627154817,879436002,0.038119,2,84
25231190,2020-04-29,2319521627154817,814766002,0.030492,2,84
25231191,2020-04-29,2319521627154817,843753004,0.040017,2,84
25211825,2020-04-29,4233235614030232,858052007,0.022864,2,84
25211826,2020-04-29,4233235614030232,852577002,0.045746,2,84
...,...,...,...,...,...,...
29567770,2020-07-28,18445692600011570228,802979001,0.022017,2,96
29567771,2020-07-28,18445692600011570228,832330003,0.016932,2,96
29567772,2020-07-28,18445692600011570228,832330003,0.016932,2,96
29567773,2020-07-28,18445692600011570228,510283001,0.010153,2,96


In [16]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 8.41 s, sys: 138 ms, total: 8.55 s
Wall time: 8.64 s


In [17]:
candidates_last_purchase


,t_dat,customer_id,article_id,price,sales_channel_id,week
25231189,2020-04-29,2319521627154817,879436002,0.038119,2,92
25231190,2020-04-29,2319521627154817,814766002,0.030492,2,92
25231191,2020-04-29,2319521627154817,843753004,0.040017,2,92
25211825,2020-04-29,4233235614030232,858052007,0.022864,2,86
25211826,2020-04-29,4233235614030232,852577002,0.045746,2,86
...,...,...,...,...,...,...
29567770,2020-07-28,18445692600011570228,802979001,0.022017,2,97
29567771,2020-07-28,18445692600011570228,832330003,0.016932,2,97
29567772,2020-07-28,18445692600011570228,832330003,0.016932,2,97
29567773,2020-07-28,18445692600011570228,510283001,0.010153,2,97


### Bestsellers candidates

In [18]:
mean_price = transactions.groupby(['week', 'article_id'])['price'].mean()

In [19]:
mean_price

week  article_id
84    108775044     0.008098
      111565001     0.006673
      111609001     0.009028
      120129001     0.016003
      129085001     0.016932
                      ...   
96    941005004     0.013336
      941005005     0.013542
      945995002     0.024924
      946748003     0.025043
      946748004     0.024551
Name: price, Length: 269933, dtype: float32

In [20]:
sales = transactions.groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank')

In [21]:
sales

week  article_id
84    841260011      1.0
      706016001      2.0
      720125001      3.0
      741356002      4.0
      759465001      5.0
                    ... 
96    610776002      8.0
      759871002      9.0
      895002002     10.0
      751471001     11.0
      372860001     12.0
Name: bestseller_rank, Length: 156, dtype: float64

In [22]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [23]:
bestsellers_previous_week

,week,article_id,bestseller_rank,price
0,85,841260011,1.0,0.015984
1,85,706016001,2.0,0.032441
2,85,720125001,3.0,0.032033
3,85,741356002,4.0,0.032389
4,85,759465001,5.0,0.007642
...,...,...,...,...
151,97,610776002,8.0,0.008284
152,97,759871002,9.0,0.006345
153,97,895002002,10.0,0.012359
154,97,751471001,11.0,0.033274


In [24]:
unique_transactions = transactions.groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [25]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
25231189,2020-04-29,2319521627154817,2,84
25211825,2020-04-29,4233235614030232,2,84
25225596,2020-04-29,4660285420023309,2,84
25236809,2020-04-29,6553270874922497,2,84
25227543,2020-04-29,7256976798788587,2,84
...,...,...,...,...
29602896,2020-07-28,18437287698903424427,1,96
29579216,2020-07-28,18442208713248673405,2,96
29601372,2020-07-28,18443005737395567187,1,96
29589628,2020-07-28,18443721675496509156,2,96


In [26]:
candidates_bestsellers = pd.merge(unique_transactions, bestsellers_previous_week, on='week')

In [27]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,bestseller_rank,price
0,2020-05-06,200292573348128,2,85,841260011,1.0,0.015984
1,2020-05-06,200292573348128,2,85,706016001,2.0,0.032441
2,2020-05-06,200292573348128,2,85,720125001,3.0,0.032033
3,2020-05-06,200292573348128,2,85,741356002,4.0,0.032389
4,2020-05-06,200292573348128,2,85,759465001,5.0,0.007642
...,...,...,...,...,...,...,...
12230323,2020-07-28,18445692600011570228,2,96,547780003,8.0,0.024814
12230324,2020-07-28,18445692600011570228,2,96,817354001,9.0,0.021913
12230325,2020-07-28,18445692600011570228,2,96,827968001,10.0,0.016436
12230326,2020-07-28,18445692600011570228,2,96,866731003,11.0,0.024893


In [28]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [29]:
candidates_bestsellers_test_week = pd.merge(test_set_transactions, bestsellers_previous_week, on='week')

In [30]:
candidates_bestsellers_test_week

,t_dat,customer_id,sales_channel_id,week,article_id,bestseller_rank,price
0,2020-04-29,2319521627154817,2,97,827968001,1.0,0.016591
1,2020-04-29,2319521627154817,2,97,706016003,2.0,0.033237
2,2020-04-29,2319521627154817,2,97,706016001,3.0,0.033265
3,2020-04-29,2319521627154817,2,97,760084003,4.0,0.024986
4,2020-04-29,2319521627154817,2,97,717490064,5.0,0.008299
...,...,...,...,...,...,...,...
6470611,2020-07-28,18443721675496509156,2,97,610776002,8.0,0.008284
6470612,2020-07-28,18443721675496509156,2,97,759871002,9.0,0.006345
6470613,2020-07-28,18443721675496509156,2,97,895002002,10.0,0.012359
6470614,2020-07-28,18443721675496509156,2,97,751471001,11.0,0.033274


In [31]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [32]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-05-06,200292573348128,2,85,841260011,0.015984
1,2020-05-06,200292573348128,2,85,706016001,0.032441
2,2020-05-06,200292573348128,2,85,720125001,0.032033
3,2020-05-06,200292573348128,2,85,741356002,0.032389
4,2020-05-06,200292573348128,2,85,759465001,0.007642
...,...,...,...,...,...,...
6470611,2020-07-28,18443721675496509156,2,97,610776002,0.008284
6470612,2020-07-28,18443721675496509156,2,97,759871002,0.006345
6470613,2020-07-28,18443721675496509156,2,97,895002002,0.012359
6470614,2020-07-28,18443721675496509156,2,97,751471001,0.033274


# Combining transactions and candidates / negative examples

In [33]:
transactions['purchased'] = 1

In [34]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

data.purchased.mean()

0.16033544522580703

In [35]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
25231189,2020-04-29,2319521627154817,879436002,0.038119,2,84,1.0
25231190,2020-04-29,2319521627154817,814766002,0.030492,2,84,1.0
25231191,2020-04-29,2319521627154817,843753004,0.040017,2,84,1.0
25211825,2020-04-29,4233235614030232,858052007,0.022864,2,84,1.0
25211826,2020-04-29,4233235614030232,852577002,0.045746,2,84,1.0
...,...,...,...,...,...,...,...
6470611,2020-07-28,18443721675496509156,610776002,0.008284,2,97,0.0
6470612,2020-07-28,18443721675496509156,759871002,0.006345,2,97,0.0
6470613,2020-07-28,18443721675496509156,895002002,0.012359,2,97,0.0
6470614,2020-07-28,18443721675496509156,751471001,0.033274,2,97,0.0


### Add bestseller information

In [36]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [37]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank
0,2020-04-29,2319521627154817,879436002,0.038119,2,84,1.0,NaN
1,2020-04-29,2319521627154817,814766002,0.030492,2,84,1.0,NaN
2,2020-04-29,2319521627154817,843753004,0.040017,2,84,1.0,NaN
3,2020-04-29,4233235614030232,858052007,0.022864,2,84,1.0,NaN
4,2020-04-29,4233235614030232,852577002,0.045746,2,84,1.0,NaN
...,...,...,...,...,...,...,...,...
27528543,2020-07-28,18443721675496509156,610776002,0.008284,2,97,0.0,8.0
27528544,2020-07-28,18443721675496509156,759871002,0.006345,2,97,0.0,9.0
27528545,2020-07-28,18443721675496509156,895002002,0.012359,2,97,0.0,10.0
27528546,2020-07-28,18443721675496509156,751471001,0.033274,2,97,0.0,11.0


In [38]:
data = data[data.week != data.week.min()].copy()
data.bestseller_rank.fillna(999, inplace=True)

In [39]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [40]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [41]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2020-05-06,200292573348128,816737001,0.050831,2,85,1.0,999.0,816737,15147,...,37,1016,11,14166,1,1,0,1,25,63947
1,2020-05-06,200292573348128,861583002,0.050831,2,85,1.0,999.0,861583,4880,...,0,1009,5,6293,1,1,0,1,25,63947
2,2020-05-01,200292573348128,864978002,0.019814,2,85,0.0,999.0,864978,18272,...,38,1003,3,16618,1,1,0,1,25,63947
3,2020-05-01,200292573348128,839311002,0.045746,2,85,0.0,999.0,839311,7596,...,0,1009,5,17611,1,1,0,1,25,63947
4,2020-05-01,200292573348128,816563001,0.022864,2,85,0.0,999.0,816563,2673,...,0,1009,5,1196,1,1,0,1,25,63947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27315060,2020-05-02,18446737527580148316,610776002,0.008284,2,97,0.0,8.0,610776,46,...,30,1002,2,60,1,1,0,1,60,96323
27315061,2020-05-02,18446737527580148316,759871002,0.006345,2,97,0.0,9.0,759871,162,...,52,1002,2,269,1,1,0,1,60,96323
27315062,2020-05-02,18446737527580148316,895002002,0.012359,2,97,0.0,10.0,895002,20538,...,0,1005,0,13785,1,1,0,1,60,96323
27315063,2020-05-02,18446737527580148316,751471001,0.033274,2,97,0.0,11.0,751471,111,...,0,1009,5,196,1,1,0,1,60,96323


In [42]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [43]:
train

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2020-05-06,200292573348128,816737001,0.050831,2,85,1.0,999.0,816737,15147,...,37,1016,11,14166,1,1,0,1,25,63947
1,2020-05-06,200292573348128,861583002,0.050831,2,85,1.0,999.0,861583,4880,...,0,1009,5,6293,1,1,0,1,25,63947
2,2020-05-01,200292573348128,864978002,0.019814,2,85,0.0,999.0,864978,18272,...,38,1003,3,16618,1,1,0,1,25,63947
3,2020-05-01,200292573348128,839311002,0.045746,2,85,0.0,999.0,839311,7596,...,0,1009,5,17611,1,1,0,1,25,63947
4,2020-05-01,200292573348128,816563001,0.022864,2,85,0.0,999.0,816563,2673,...,0,1009,5,1196,1,1,0,1,25,63947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18728852,2020-07-24,18446522881501826039,547780003,0.024814,1,96,0.0,8.0,547780,1272,...,27,1002,2,271,-1,-1,0,0,18,0
18728853,2020-07-24,18446522881501826039,817354001,0.021913,1,96,0.0,9.0,817354,2747,...,0,1008,20,23341,-1,-1,0,0,18,0
18728854,2020-07-24,18446522881501826039,827968001,0.016436,1,96,0.0,10.0,827968,1066,...,30,1002,2,1227,-1,-1,0,0,18,0
18728855,2020-07-24,18446522881501826039,866731003,0.024893,1,96,0.0,11.0,866731,3609,...,21,1005,0,3130,-1,-1,0,0,18,0


In [44]:
test

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
18728857,2020-07-26,28847241659200,887770001,0.016932,1,97,0.0,999.0,887770,727,...,10,1010,6,3692,1,1,0,1,21,57896
18728858,2020-05-13,28847241659200,827968001,0.016591,2,97,0.0,1.0,827968,1066,...,30,1002,2,1227,1,1,0,1,21,57896
18728859,2020-05-13,28847241659200,706016003,0.033237,2,97,0.0,2.0,706016,172,...,1,1009,5,30,1,1,0,1,21,57896
18728860,2020-05-13,28847241659200,706016001,0.033265,2,97,0.0,3.0,706016,172,...,1,1009,5,30,1,1,0,1,21,57896
18728861,2020-05-13,28847241659200,760084003,0.024986,2,97,0.0,4.0,760084,1134,...,1,1009,5,847,1,1,0,1,21,57896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27315060,2020-05-02,18446737527580148316,610776002,0.008284,2,97,0.0,8.0,610776,46,...,30,1002,2,60,1,1,0,1,60,96323
27315061,2020-05-02,18446737527580148316,759871002,0.006345,2,97,0.0,9.0,759871,162,...,52,1002,2,269,1,1,0,1,60,96323
27315062,2020-05-02,18446737527580148316,895002002,0.012359,2,97,0.0,10.0,895002,20538,...,0,1005,0,13785,1,1,0,1,60,96323
27315063,2020-05-02,18446737527580148316,751471001,0.033274,2,97,0.0,11.0,751471,111,...,0,1009,5,196,1,1,0,1,60,96323


In [45]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [46]:
train_baskets

array([20, 23, 27, ..., 14, 27, 13])

In [47]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [48]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 556 ms, sys: 1.54 s, total: 2.1 s
Wall time: 2.56 s


In [49]:
train_X

,article_id,product_type_no,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,bestseller_rank
0,816737001,272,1010023,9,4,5,1772,1,2,57,1016,1,1,0,1,25,63947,999.0
1,861583002,272,1010023,9,4,5,1722,0,1,15,1009,1,1,0,1,25,63947,999.0
2,864978002,254,1010016,21,3,8,5963,1,2,58,1003,1,1,0,1,25,63947,999.0
3,839311002,272,1010016,42,5,18,1722,0,1,15,1009,1,1,0,1,25,63947,999.0
4,816563001,272,1010016,9,4,5,1722,0,1,15,1009,1,1,0,1,25,63947,999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18728852,547780003,272,1010010,7,1,12,1643,1,2,51,1002,-1,-1,0,0,18,0,8.0
18728853,817354001,264,1010016,19,4,11,1222,0,1,15,1008,-1,-1,0,0,18,0,9.0
18728854,827968001,255,1010016,10,3,9,1676,0,1,16,1002,-1,-1,0,0,18,0,10.0
18728855,866731003,273,1010016,12,1,11,8310,9,26,5,1005,-1,-1,0,0,18,0,11.0


In [50]:
train_y

0           1.0
1           1.0
2           0.0
3           0.0
4           0.0
           ... 
18728852    0.0
18728853    0.0
18728854    0.0
18728855    0.0
18728856    0.0
Name: purchased, Length: 18728857, dtype: float64

# Model training

In [51]:
from lightgbm.sklearn import LGBMRanker

In [52]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [53]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.852750
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.146089
[LightGBM] [Debug] init for col-wise cost 0.243824 seconds, init for row-wise cost 1.194251 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.438435 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1071
[LightGBM] [Info] Number of data points in the train set: 18728857, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
CPU times: user 45.3 s, sys: 9.74 s, total: 55.1 s
Wall time: 17 s


In [ ]:
"""
import pickle5 as pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(ranker, f)
"""

In [54]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9990815346764488
age 0.0004093333688474438
club_member_status 0.0001218782258333133
section_no 0.0001132751182529611
postal_code 7.922175501506838e-05
department_no 7.403034496623365e-05
Active 4.172483127224935e-05
article_id 3.269549711550455e-05
product_type_no 2.623788212466541e-05
fashion_news_frequency 2.0068300123739762e-05
garment_group_no 0.0
FN 0.0
index_code 0.0
perceived_colour_master_id 0.0
perceived_colour_value_id 0.0
colour_group_code 0.0
graphical_appearance_no 0.0
index_group_no 0.0


# Calculate predictions

In [56]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test.sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = ['0' + str(a_id) for a_id in \
                             bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [57]:
test['preds']

18728857    0.165661
18728858   -0.192690
18728859   -0.190290
18728860   -0.190290
18728861   -0.182170
              ...   
27315060   -0.194692
27315061   -0.194692
27315062   -0.196595
27315063   -0.194692
27315064   -0.194692
Name: preds, Length: 8359062, dtype: float64

In [58]:
c_id2predicted_article_ids

{28847241659200: [887770001,
  760084003,
  759871002,
  706016003,
  706016001,
  730683050,
  372860002,
  372860001,
  827968001,
  717490064,
  610776002,
  751471001,
  895002002],
 41318098387474: [858918004,
  688728023,
  842605012,
  827968001,
  706016003,
  706016001,
  760084003,
  717490064,
  730683050,
  372860002,
  610776002,
  759871002,
  751471001,
  372860001,
  895002002],
 77117344919861: [881759005,
  599580061,
  844294002,
  599580068,
  844294001,
  827968001,
  706016003,
  706016001,
  760084003,
  717490064,
  730683050,
  372860002,
  610776002,
  759871002,
  751471001,
  372860001,
  895002002],
 200292573348128: [880777001,
  784332002,
  599580086,
  688537029,
  827968001,
  706016003,
  706016001,
  760084003,
  717490064,
  730683050,
  372860002,
  610776002,
  759871002,
  751471001,
  372860001,
  895002002],
 208119717816961: [540334001,
  572797049,
  572797002,
  859105007,
  827968001,
  706016003,
  706016001,
  760084003,
  717490064,
  73

# Create a submission

In [59]:
sub = pd.read_csv('data/sample_submission.csv')

In [60]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    if c_id in c_id2predicted_article_ids:
        ps = ['0' + str(p) for p in c_id2predicted_article_ids[c_id]]
        preds.append(ps)
    else:
        preds.append(bestsellers_last_week)

CPU times: user 5.48 s, sys: 1.04 s, total: 6.51 s
Wall time: 6.88 s


In [61]:
preds

[['0827968001',
  '0706016003',
  '0706016001',
  '0760084003',
  '0717490064',
  '0730683050',
  '0372860002',
  '0610776002',
  '0759871002',
  '0895002002',
  '0751471001',
  '0372860001'],
 ['0826211002',
  '0827968001',
  '0706016003',
  '0706016001',
  '0760084003',
  '0717490064',
  '0730683050',
  '0372860002',
  '0610776002',
  '0759871002',
  '0751471001',
  '0372860001',
  '0895002002'],
 ['0827968001',
  '0706016003',
  '0706016001',
  '0760084003',
  '0717490064',
  '0730683050',
  '0372860002',
  '0610776002',
  '0759871002',
  '0895002002',
  '0751471001',
  '0372860001'],
 ['0827968001',
  '0706016003',
  '0706016001',
  '0760084003',
  '0717490064',
  '0730683050',
  '0372860002',
  '0610776002',
  '0759871002',
  '0895002002',
  '0751471001',
  '0372860001'],
 ['0589440005',
  '0827971001',
  '0818320001',
  '0827968001',
  '0706016003',
  '0706016001',
  '0760084003',
  '0717490064',
  '0730683050',
  '0372860002',
  '0610776002',
  '0759871002',
  '0751471001',
  '0

In [62]:
preds = [' '.join(ps) for ps in preds]
sub.prediction = preds

In [63]:
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0827968001 0706016003 0706016001 0760084003 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0827968001 0706016003 0706016001 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0827968001 0706016003 0706016001 0760084003 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0827968001 0706016003 0706016001 0760084003 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0589440005 0827971001 0818320001 0827968001 07...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0739590033 0832321002 0832320002 0860949003 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0805370005 0668012013 0685816044 0760084003 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0636323002 0835561003 0795975009 0760084003 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0882810001 0760084003 0759871002 0706016003 07...


In [64]:
sub_name = 'basic_with_bestseller_information'
sub.to_csv(f'data/{sub_name}.csv.gz', index=False)

In [ ]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f 'data/subs/{sub_name}.csv.gz' -m {sub_name}

In [118]:
with open("basic_with_bestseller_information.csv", "rb") as f:
    df = pd.read_csv(f)

In [119]:
df

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0827968001 0706016003 0706016001 0760084003 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0827968001 0706016003 0706016001 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0827968001 0706016003 0706016001 0760084003 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0827968001 0706016003 0706016001 0760084003 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0589440005 0827971001 0818320001 0827968001 07...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0739590033 0832321002 0832320002 0860949003 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0805370005 0668012013 0685816044 0760084003 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0636323002 0835561003 0795975009 0760084003 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0882810001 0760084003 0759871002 0706016003 07...


In [123]:
customer_id = "0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa"
pred = df.loc[df['customer_id'] == customer_id, 'prediction'].iloc[0]
pred


'0826211002 0827968001 0706016003 0706016001 0760084003 0717490064 0730683050 0372860002 0610776002 0759871002 0751471001 0372860001 0895002002'

# Where to go from here

This notebook has all the components of a valid solution.

We generate negative examples.
We generate candidates for the test set.
We also enrich transaction records with calculated statistics (bestseller information from previous week).

From there on we train a model and use that model to output predictions.

But all of this is intentionally very basic. Look how many components we had to get in place to get to this spot!

Now the fun part begins.

Further work on this should fall into three categories IMHO:
* use functionality from this and the `Solution Warmup` notebook to work out a good way of validating the LGBM ranker and end-to-end predictions (all the components for this have been developed, they just need to be piece together)
* develop useful features that would describe (Kaggle kernels for this competition already have a lot of code you can reuse, everything by Chris Deotte is gold)
    * the customer
    * the article
    * customer tendencies
* train a couple of varied models and figure out how to blend them